In [1]:
!pip install ultralytics
!pip install -U ipywidgets

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/33/b1/f2cc216c80bee008db680d78026ee1a0c5421755f4894c8ec27dc773fbe0/ultralytics-8.1.4-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.5/701.5 kB 23.8 MB/s eta 0:00:00
  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/4a/0e/57ed498fafbc60419a9332d872e929879ceba2d73cb11d284d7112472b3e/ipywidgets-8.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for widgetsnbextension~=4.0.9 from https://files.pythonhosted.org/packages/29/03/107d96077c4befed191f7ad1a12c7b52a8f9d2778a5836d59f9855c105f6/widgetsnbextension-4.0.9-py3-none-any.whl.metadata
  Obtaining dependency information for jupyterlab-widgets~=3.0.9 from https://files.pythonhosted.org/packages/e8/05/0ebab152288693b5ec7b339aab857362947031143b282853b4c2dd4b5b40/jupyterlab_widgets-3.0

In [2]:
!mkdir -p /kaggle/temp/train/labels
!mkdir -p /kaggle/temp/train/images

!mkdir -p /kaggle/temp/val/labels
!mkdir -p /kaggle/temp/val/images

In [3]:
!cp /kaggle/input/kidney-1-dense-to-yolo/train/labels/* /kaggle/temp/train/labels
!cp /kaggle/input/kidney-1-dense-to-yolo/train/images/* /kaggle/temp/train/images

!cp /kaggle/input/kidney-1-voi-to-yolo/train/labels/* /kaggle/temp/train/labels
!cp /kaggle/input/kidney-1-voi-to-yolo/train/images/* /kaggle/temp/train/images

!cp /kaggle/input/kidney-2-to-yolo/train/labels/* /kaggle/temp/train/labels
!cp /kaggle/input/kidney-2-to-yolo/train/images/* /kaggle/temp/train/images

!cp /kaggle/input/kidney-3-sparse-to-yolo/train/labels/* /kaggle/temp/train/labels
!cp /kaggle/input/kidney-3-sparse-to-yolo/train/images/* /kaggle/temp/train/images

In [4]:
!cp /kaggle/input/kidney-3-dense-to-yolo/train/labels/* /kaggle/temp/val/labels
!cp /kaggle/input/kidney-3-dense-to-yolo/train/images/* /kaggle/temp/val/images

In [5]:
# !tail /kaggle/input/kidney-1-voi-to-yolo/train/kidney_1_voi.json

# Write yolo dataset yaml

In [6]:
import yaml

In [7]:
!mkdir train

In [8]:
train_path = '/kaggle/temp/train'
val_path = '/kaggle/temp/val'
names = {0: 'Vessel'}
yaml_path = 'train/train.yaml'

# Number of classes
nc = len(names)

# Create a dictionary with the required content
yaml_data = {
    'names': names,
    'nc': nc,
    'test': '',
    'train': train_path,
    'val': val_path
}

# Write the dictionary to a YAML file
with open(yaml_path, 'w') as file:
    yaml.dump(yaml_data, file, default_flow_style=False)

In [9]:
!cat {yaml_path}

names:
  0: Vessel
nc: 1
test: ''
train: /kaggle/temp/train
val: /kaggle/temp/val


In [10]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("wandb-api-key")
wandb.login(key=wandb_api_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
from ultralytics import YOLO

model = YOLO('yolov8n-seg.pt')

100%|██████████| 6.73M/6.73M [00:00<00:00, 99.7MB/s]


In [12]:
project = 'hacking_human_vasculature'
name = 'kidney_3_dense_val_rest_train_yolo_s_1600_9'

results = model.train(data=yaml_path,
                      project=project,
                      name=name,
                      epochs=100,
                      time=3,
                      patience=10, 
                      batch=9,
                      imgsz=1600,
                      cache=True
                     )

Ultralytics YOLOv8.1.4 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=train/train.yaml, epochs=100, time=3, patience=10, batch=9, imgsz=1600, save=True, save_period=-1, cache=True, device=None, workers=8, project=hacking_human_vasculature, name=kidney_3_dense_val_rest_train_yolo_s_1600_9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, s

100%|██████████| 755k/755k [00:00<00:00, 22.3MB/s]
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

wandb: Currently logged in as: aaalex-lit. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.2
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240122_231448-51k9z9y3
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run kidney_3_dense_val_rest_train_yolo_s_1600_9
wandb: ⭐️ View project at https://wandb.ai/aaalex-lit/hacking_human_vasculature
wandb: 🚀 View run at https://wandb.ai/aaalex-lit/hacking_human_vasculature/runs/51k9z9y3


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   9%|▊         | 5/58 [03:23<1:00:22, 68.35s/it]

WARNING ⚠️ NMS time limit 2.900s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 58/58 [04:46<00:00,  4.93s/it]


                   all       1035      41540      0.514      0.423      0.315      0.181      0.442      0.394      0.303      0.156
Speed: 5.5ms preprocess, 21.1ms inference, 0.0ms loss, 7.2ms postprocess per image
Results saved to hacking_human_vasculature/kidney_3_dense_val_rest_train_yolo_s_1600_9


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▃▆█▇▆▅▄▃▂▁
wandb:                  lr/pg1 ▃▆█▇▆▅▄▃▂▁
wandb:                  lr/pg2 ▃▆█▇▆▅▄▃▂▁
wandb:        metrics/mAP50(B) ▁▆▇▆▇█████
wandb:        metrics/mAP50(M) ▁▆▆▆▇▇████
wandb:     metrics/mAP50-95(B) ▁▆▆▆▇█████
wandb:     metrics/mAP50-95(M) ▁▆▆▆▇▇▇▇██
wandb:    metrics/precision(B) █▁▂▁▃▄▄▅▅▄
wandb:    metrics/precision(M) █▁▁▂▂▂▃▄▄▄
wandb:       metrics/recall(B) ▁▇▇▇██████
wandb:       metrics/recall(M) ▁▇▇▇▇▇████
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▆▆▅▄▃▂▂▁▁
wandb:          train/cls_loss █▃▃▂▂▂▁▁▁▁
wandb:          train/dfl_loss █▄▄▄▃▂▂▂▁▁
wandb:          train/seg_loss █▅▄▃▂▂▂▁▁▁
wandb:            val/box_loss █▃▂▂▂▂▁▂▁▁
wandb:            val/cls_loss ▁▅▆▄▅▅▇▇██
wandb:            val/dfl_loss █▃▃▃▂▂▂▂▁▁
wandb:            val/seg_loss

In [13]:
wandb.finish()